# Проект № 4. Мои университеты это уфимские кофейни.


В этом проекте я постараюсь изобразить и понять пространственное расположение кофеен в городе Уфа.  


## Поехали


код был собрал в одну кодовую ячейку, но надеюсь, благодаря пояснениям внутри, все будет понятно.


In [35]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster, MiniMap, MousePosition, Fullscreen
import branca.colormap as cm
from shapely.geometry import Point
import numpy as np  


# Настройка OSMnx
ox.settings.log_console = True
ox.settings.timeout = 300
ox.settings.cache_folder = "./cache"

# 1. Функция для загрузки административных границ с населением
def get_ufa_districts():
    print("Загрузка административных границ...")
    
    ufa_city = ox.geocode_to_gdf("Уфа, Россия")
    
    required_districts = {
        "Дёмский район": 74701,
        "Калининский район": 209000,
        "Кировский район": 164000,
        "Ленинский район": 94758,
        "Октябрьский район": 246161,
        "Орджоникидзевский район": 169798,
        "Советский район": 178000
    }
    
    districts = ox.features_from_place(
        "Уфа, Россия",
        tags={'admin_level': '9', 'boundary': 'administrative'}
    )
    
    districts = districts[districts['name'].isin(required_districts.keys())]
    districts['population'] = districts['name'].map(required_districts)
    
    return ufa_city, districts

# 2. Функция для загрузки POI
def get_pois(districts_gdf, tags, name_col=None):
    print(f"\nЗагрузка {tags}...")
    
    all_pois = gpd.GeoDataFrame()
    
    for _, district in districts_gdf.iterrows():
        try:
            pois = ox.features_from_polygon(
                district['geometry'],
                tags=tags
            )
            
            if not pois.empty:
                if name_col:
                    pois = pois[pois[name_col].notna()]
                pois['district'] = district['name']
                all_pois = pd.concat([all_pois, pois])
                
        except Exception as e:
            print(f"Ошибка в районе {district['name']}: {e}")
    
    return all_pois


# Добавляем новую функцию для загрузки университетов с координатами
def get_ufa_universities():
    universities_data = [
        {"name": "БашГУ", "address": "г. Уфа, ул. Заки Валиди, 32", "lon": 55.933422 , "lat": 54.720714},
        {"name": "УГАТУ", "address": "г. Уфа, ул. Карла Маркса, 12", "lon": 55.942450, "lat": 54.724989},
        {"name": "БГМУ", "address": "г. Уфа, ул. Ленина, 3", "lon": 55.948468, "lat": 54.727448},
        {"name": "УГНТУ", "address": "г. Уфа, ул. Космонавтов, 1", "lon": 56.058323, "lat": 54.818558},
        {"name": "БГПУ", "address": "г. Уфа, ул. Октябрьской революции, 3а", "lon": 55.947741, "lat": 54.724609},
        {"name": "УЮИ МВД РФ", "address": "г. Уфа, ул. Муксинова, 2", "lon": 55.996214, "lat": 54.693712},
        {"name": "БАГСУ", "address": "г. Уфа, ул. Заки Валиди, 40", "lon": 55.946339, "lat": 54.717792},
        {"name": "Уфимский филиал РЭУ", "address": "г. Уфа, ул. Чернышевского, 49", "lon": 55.935640, "lat": 54.731051},
        {"name": "БГАУ", "address": "г. Уфа, ул. 50-летия Октября, 34", "lon": 55.983503, "lat": 54.738417},
        {"name": "УГИИ", "address": "г. Уфа, ул. Ленина, 14", "lon": 55.946016, "lat": 54.722124}
    ]
    
    universities = []
    for uni in universities_data:
        universities.append({
            "name": uni["name"],
            "address": uni["address"],
            "geometry": Point(uni["lon"], uni["lat"])  # Теперь Point будет определен
        })
    
    return gpd.GeoDataFrame(universities, geometry="geometry", crs="EPSG:4326")

# Функция для анализа пространственной зависимости
def analyze_spatial_relationship(coffee_points, universities):
    # Рассчитываем расстояние от каждой кофейни до ближайшего университета
    coffee_points["nearest_univ"] = np.nan
    coffee_points["dist_to_univ"] = np.nan
    
    for idx, coffee in coffee_points.iterrows():
        distances = universities.geometry.distance(coffee.geometry)
        nearest_idx = distances.idxmin()
        coffee_points.at[idx, "nearest_univ"] = universities.at[nearest_idx, "name"]
        coffee_points.at[idx, "dist_to_univ"] = distances.min()
    
    # Конвертируем метры в километры
    coffee_points["dist_to_univ_km"] = coffee_points["dist_to_univ"] / 1000
    
    # Анализ плотности
    bins = [0, 0.2, 0.5, 1, 2, 5]
    coffee_points["dist_category"] = pd.cut(
        coffee_points["dist_to_univ_km"],
        bins=bins,
        labels=[f"<{b} км" for b in bins[1:]]
    )
    
    return coffee_points


# Функция создания карты
def create_interactive_map(ufa_city, districts, coffee_pois, universities):
    print("\nСоздание интерактивной карты...")
    
    # Фильтруем только точечные объекты
    coffee_points = coffee_pois[coffee_pois.geometry.type == 'Point'].copy()
    
    # Анализируем пространственную зависимость
    coffee_with_dist = analyze_spatial_relationship(coffee_points, universities)
    
    # Центрируем карту на Уфе
    centroid = ufa_city.geometry.centroid
    m = folium.Map(
        location=[centroid.y, centroid.x],
        zoom_start=12,
        tiles="cartodbpositron",
        control_scale=True
    )
    
    # Создаем FeatureGroup для каждого слоя
    coffee_distance_layer = folium.FeatureGroup(name="Кофейни в зависимости от расстояния до университетов", show=True)
    coffee_circles = folium.FeatureGroup(name="Население района", show=False)
    coffee_cluster = folium.FeatureGroup(name="Кластеры кофеен", show=True)
    univ_layer = folium.FeatureGroup(name="Университеты", show=True)
    buffer_layer = folium.FeatureGroup(name="Зоны 500м от университетов  ", show=False)
    heatmap_layer = folium.FeatureGroup(name="Тепловая карта кофеен", show=False)
    
    # 1. Слой районов (Choropleth)
    districts['coffee_count'] = districts['name'].map(
        coffee_points.groupby('district').size()
    )
    districts['coffee_per_100k'] = (districts['coffee_count'] / 
                                   districts['population'] * 100000).fillna(0)
    
    colormap = cm.LinearColormap(
        colors=['green', 'yellow', 'red'],
        vmin=districts['coffee_per_100k'].min(),
        vmax=districts['coffee_per_100k'].max()
    )
    
    style_function = lambda x: {
        'fillColor': colormap(x['properties']['coffee_per_100k']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.45
    }
    
    folium.GeoJson(
        districts,
        name="Плотность кофеен в районах (кофейни /100 тыс. населения)",
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['name', 'population', 'coffee_count', 'coffee_per_100k'],
            aliases=['Район', 'Население', 'Кофеен', 'Кофеен/100к'],
            localize=True
        )
    ).add_to(m)
    
    # 2. Добавляем университеты с буферными зонами
    for idx, uni in universities.iterrows():
        # Маркер университета
        folium.Marker(
            location=[uni.geometry.y, uni.geometry.x],
            popup=f"<b>{uni['name']}</b><br>{uni['address']}",
            icon=folium.Icon(color="blue", icon="university", prefix="fa")
        ).add_to(univ_layer)
        
        # Буферная зона 500м
        folium.Circle(
            location=[uni.geometry.y, uni.geometry.x],
            radius=500,
            color="#3186cc",
            fill=True,
            fill_opacity=0.2,
            popup=f"{uni['name']} (500м зона)"
        ).add_to(buffer_layer)
    
    # 3 Добавляем кофейни с цветовой кодировкой по расстоянию
    if not coffee_with_dist.empty:
        max_dist = coffee_with_dist["dist_to_univ_km"].max()
        
        for idx, row in coffee_with_dist.iterrows():
            # Цвет от зеленого (близко) к красному (далеко)
            dist_ratio = row["dist_to_univ_km"] / max_dist
            color = f"rgb({int(255*dist_ratio)}, {int(255*(1-dist_ratio))}, 0)"
            
            popup_text = f"""
            <b>{row.get('name', 'Кофейня')}</b><br>
            Район: {row['district']}<br>
            До {row['nearest_univ']}: {row['dist_to_univ_km']:.2f} км<br>
            Адрес: {row.get('addr:street', 'нет данных')}
            """
            
            folium.CircleMarker(
                location=[row.geometry.y, row.geometry.x],
                radius=5,
                popup=folium.Popup(popup_text, max_width=250),
                color=color,
                fill=True,
                fill_color=color
            ).add_to(coffee_distance_layer)
        
        # Круги (размер зависит от населения района)
        for district_name, group in coffee_points.groupby('district'):
            district_pop = districts[districts['name'] == district_name]['population'].values[0]
            radius = district_pop / 10000  # Масштабируем радиус
            
            folium.CircleMarker(
                location=[group.geometry.y.mean(), group.geometry.x.mean()],
                radius=radius,
                popup=f"{district_name}: {len(group)} кофеен",
                color='crimson',
                fill=True,
                fill_color='crimson'
            ).add_to(coffee_circles)
        
        # Кластеры для группировки маркеров
        marker_cluster = MarkerCluster()
        for idx, row in coffee_points.iterrows():
            marker_cluster.add_child(
                folium.Marker([row.geometry.y, row.geometry.x])
            )
        coffee_cluster.add_child(marker_cluster)
    
        
        # Тепловая карта
        heat_data = [[row.geometry.y, row.geometry.x] for _, row in coffee_with_dist.iterrows()]
        HeatMap(heat_data, radius=15).add_to(heatmap_layer)
        
        # Легенда расстояний кофеен от университетов        
        legend_html = '''
        <div style="
            position: absolute;
            bottom: 20px;
            left: 20px;
            width: 220px;
            background: white;
            border: 1px solid #ccc;
            border-radius: 5px;
            padding: 10px;
            font-family: Arial;
            z-index: 1000;
            box-shadow: 0 0 5px rgba(0,0,0,0.2);
        ">
            <h4 style="margin: 0 0 8px 0; font-size: 12px; text-align: center;">
                Расстояние кофеен до университетов (км)
            </h4>
            
            <!-- Цветовая шкала -->
            <div style="background: linear-gradient(to right, #00ff00, #ff0000); 
                        height: 12px;
                        margin-bottom: 5px;
                        border-radius: 3px;
                        position: relative;">
            </div>
            
            <!-- Деления и подписи -->
            <div style="display: flex; justify-content: space-between; 
                        position: relative; height: 20px;
                        margin-top: -5px;">
                <div style="position: absolute; left: 0%; transform: translateX(-50%);">
                    <div style="width: 1px; height: 5px; background: #333;"></div>
                    <div style="font-size: 11px; text-align: center; margin-top: 2px;">0</div>
                </div>
                <div style="position: absolute; left: 25%; transform: translateX(-50%);">
                    <div style="width: 1px; height: 5px; background: #333;"></div>
                    <div style="font-size: 11px; text-align: center; margin-top: 2px;">0.5</div>
                </div>
                <div style="position: absolute; left: 50%; transform: translateX(-50%);">
                    <div style="width: 1px; height: 5px; background: #333;"></div>
                    <div style="font-size: 11px; text-align: center; margin-top: 2px;">1</div>
                </div>
                <div style="position: absolute; left: 75%; transform: translateX(-50%);">
                    <div style="width: 1px; height: 5px; background: #333;"></div>
                    <div style="font-size: 11px; text-align: center; margin-top: 2px;">2</div>
                </div>
                <div style="position: absolute; left: 100%; transform: translateX(-50%);">
                    <div style="width: 1px; height: 5px; background: #333;"></div>
                    <div style="font-size: 11px; text-align: center; margin-top: 2px;">5</div>
                </div>
            </div>
            
            <!-- Подписи краев -->
            <div style="display: flex; justify-content: space-between; font-size: 12px; margin-top: 5px;">
                <span>Близко</span>
                <span>Далеко</span>
            </div>
        </div>
        '''

    # Добавляем легенду на карту
    m.get_root().html.add_child(folium.Element(legend_html))

  
    # Добавляем все слои на карту
    coffee_distance_layer.add_to(m)
    coffee_circles.add_to(m)
    coffee_cluster.add_to(m)
    univ_layer.add_to(m)
    buffer_layer.add_to(m)
    heatmap_layer.add_to(m)
        
    # Добавляем элементы управления
    folium.LayerControl(collapsed=False).add_to(m)
    Fullscreen().add_to(m)
    MiniMap().add_to(m)
    MousePosition().add_to(m)
    
    # Сохраняем карту
    m.save("ufa_coffee_map.html")
    print("Карта сохранена как 'ufa_coffee_map.html'")
    
    return m

# Мейн скрипт
if __name__ == "__main__":
    # 1. Загружаем границы районов
    ufa_city, ufa_districts = get_ufa_districts()
    
    # 2. Загружаем кофейни
    coffee_pois = get_pois(
        ufa_districts, 
        tags={'amenity': 'cafe', 'cuisine': 'coffee_shop'},
        name_col='name'
    )
    
    # 3. Загружаем университеты
    universities = get_ufa_universities()
    
    # 5. Создаем интерактивную карту
    create_interactive_map(ufa_city, ufa_districts, coffee_pois, universities)

Загрузка административных границ...

Загрузка {'amenity': 'cafe', 'cuisine': 'coffee_shop'}...

Создание интерактивной карты...


C:\Users\1\AppData\Local\Temp\ipykernel_6432\1678417342.py:99: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = universities.geometry.distance(coffee.geometry)
C:\Users\1\AppData\Local\Temp\ipykernel_6432\1678417342.py:129: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = ufa_city.geometry.centroid
c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\folium\utilities.py:103: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
c:\Users\1\AppData\Local\Programs\Python\Python311\Lib\site-packages\folium\utilities.py:109: FutureWarning: Calling float on a single element Series

Карта сохранена как 'ufa_coffee_map.html'


# Вот и все! Подробное объяснение в файле READ.me, результаты - в html карте. Спасибо!
